# Data visualization ― week 5, part I

![](images/HometoWork_spread1.jpg)

# Agenda

- Introduction to geospatial representations
- Building a simple GIS with Turtle
- Python packages for geospatial analyses — overview

# History of geospatial representations: Dr Snow's studies of cholera spread in 19th century London

![](images/SnowMap_Points.png)

Go to the video: https://www.youtube.com/watch?v=Pq32LB8j2K8

# What are the methodological roots of geo-spatial analyses?

- GIS
- remote sensoring
- elevation data

# Geographical information systems

- computer mapping evolved with the computer itself in the 1960s
- the term GIS began with the Canadian Department of Forestry and Rural Development
    - Dr. Roger Tomlinson headed a team of 40 developers in an agreement with IBM to build the Canada Geographic Information System (CGIS)
    - the CGIS tracked the natural resources of Canada and allowed the profiling of these features for further analysis
    - the CGIS stored each type of land cover as a different layer

# Example of GIS ― cattles sent to Paris from all around France

![](images/example_of_gis.png)

Source: Minard (early 1900s)

# Remote sensoring

- 'remote sensing' is the collection of information about an object without making physical contact with that object
- in the context of geospatial analysis, the object is usually the Earth
- by extracting features from images, modern remote sensing technologies have enabled GIS

# Early history of remote sensoring

![](images/56914f0d6cbe4.jpeg)

Aerial picture taken in 1860 over the city of Boston

# Application of remote sensoring in military intelligence: The battle for the space

![](images/Sputnik-695836.jpg)

# Elevation data

- a Digital Elevation Model (DEM) is a three-dimensional representation of a planet's terrain (e.g., the Earth)
- before computers, representations of elevation data were limited to topographic maps created through traditional land surveys
- technology existed to create three-dimensional models from stereoscopic images or physical models from materials such as clay or wood


![](images/worlddem_south_province_iceland_2012.jpg)

# The central role of geospatial analysis in modern data science

- geospatial analysis helps people make better decisions
    - it can answer critical questions that are at the heart of the choice to be made and often cannot be answered any other way. Until recentlygeospatial
- technology and data were tools available only to governments and well-funded researchers
    - however, in the last decade, data has become much more widely available


# Thematic maps

# What are thematic maps?

Thematic maps visually represents features as they relate geographically for navigation or
planning

![](images/PMC4852634_rsos150691-g3.png)

# Why/how do we use thematic maps?

- thematic maps are designed for a targeted audience to answer specific questions
- the value of thematic maps lies in what they do not show
    - geographic features should be minimized to avoid distracting the reader from the theme
    - most thematic maps include political boundaries such as country or state borders but omit navigational features (e.g., street names or points of interest)


# Rendering

- geographic data including points, lines, and polygons
- these artifacts are stored numerically as one or more points, which come in (x,y) pairs or (x,y,z) tuples


# Geographic and screen coordinates

 - geographic data are stored in a coordinate system representing a grid overlaid on the Earth, which is three-dimensional and round
 - screen coordinates, also known as pixel coordinates, represent a grid of pixels on a flat, two-dimensional computer screen
 - mapping x and y world coordinates to pixel coordinates is fairly straightforward and involves a simple scaling algorithm
 - however, if a z coordinate exists, then a more complicated transform must be performed to map coordinates from three-dimensional space to a two-dimensional plane

# Let's build a simple GIS

Steps:

- creating the data model
- creating the map renderer that draws the data

# Good reference book on this

![](https://images-na.ssl-images-amazon.com/images/I/51U5bZBX0VL._SX404_BO1,204,203,200_.jpg)

Note: the GIS example we analyze comes from this book

# Step 0: Let's leverage some drawing capabilities

Import the module Turtle

In [1]:
import turtle as t

# Step 1: Data model setup

Set up the data for a sample USA state (Colorado) as a list with name, polygon points, and population

In [2]:
NAME = 0
POINTS = 1
POP = 2

state = [
    "COLORADO",
    [
        [-109, 37], [-109, 41],
        [-102, 41], [-102, 37]
    ], 5187582
]

# Step 2: Expand data

- let's add some information about cities nested in the focal state
- the cities will be stored as nested lists 
- each city's location consists of a single point as a longitude and latitude pair

In [3]:
cities = []
cities.append(["DENVER",[-104.98, 39.74], 634265])
cities.append(["BOULDER",[-105.27, 40.02], 98889])
cities.append(["DURANGO",[-107.88,37.28], 17069])

# Step 3: Fix map size

- we will now render our GIS data as a map by first defining a map size
- the width and height can be anything that you want depending on your screen resolution

In [4]:
map_width = 400
map_height = 300

# Step 4: Scaling the map

- in order to scale the map to the graphics canvas, we must first determine the bounding box of the largest layer, which is the state
- we'll set the map's bounding box to a global scale and reduce it to the size of the state. To do so:
    - we'll loop through the longitude and latitude of each point and compare it with the current minimum and maximum x and y values
    - if it is larger than the current maximum or smaller than the current minimum, we'll make this value the new maximum or minimum, respectively

In [5]:
minx = 105
maxx = -112
miny = 40
maxy = -44

for x, y in state[POINTS]:
    if x < minx: minx = x
    elif x > maxx: maxx = x
    if y < miny: miny = y
    elif y > maxy: maxy = y

# Step 4: Scaling the map (cont'd)

- the second step to scaling is to calculate a ratio between the actual state and the tiny canvas that we will render it on
- this ratio is used for coordinate to pixel conversion. We get the size along the x and y axes of the state and then we divide the map width and height by these numbers to get our scaling ratio:

In [6]:
dist_x = maxx - minx
dist_y = maxy - miny
x_ratio = map_width / dist_x
y_ratio = map_height / dist_y

# Step 5: Sample GIS function

- be 'convert()' our sample GIS function
- we require convert() to transform a point in the map coordinates from one of our data layers to pixel coordinates using the previous calculations
- in order to to account for the unusual center origin of the turtle graphics canvas, we divide the map width and height in half and subtract it from the final conversion 

In [7]:
def convert(point):
    lon = point[0]
    lat = point[1]
    x = map_width - ((maxx - lon) * x_ratio)
    y = map_height - ((maxy - lat) * y_ratio)
    # Python turtle graphics start in the
    # middle of the screen
    # so we must offset the points so they are centered
    x = x - (map_width/2)
    y = y - (map_height/2)
    return [x,y]

# Step 6: Render our GIS as a thematic map

- the turtle module uses the concept of a cursor called a pen
- moving the cursor around the canvas is exactly the same as moving a pen around a piece of paper
- the cursor will draw a line when you move it―so, you'll notice that throughout the code, we use the t.up() and t.down() commands to pick the pen up when we want to move to a new location and put it down when we're ready to draw. We have some important steps in this section
- as the border of Colorado is a polygon, we must draw a line between the last point and first point to close the polygon. We can also leave out the closing step and just add a duplicate point to the Colorado dataset
- once we draw the state, we'll use the write() method to label the polygon

In [8]:
first_pixel = None
t.up()

for point in state[POINTS]:
    pixel = convert(point)
    if not first_pixel:
        first_pixel=pixel
    t.goto(pixel)
    t.down()


t.goto(first_pixel)
t.up()
t.goto([0,0])
t.write(state[NAME], align="center", font=("Arial",16,"bold"))

# Step 6: Render our GIS as a thematic map (cont'd)

- now, we'll render the cities as point locations and label them with their names and population;
- as the cities are a group of features in a list, we'll loop through them to render them. Instead of drawing lines by moving the pen around, we'll use the turtle dot() method to plot a small circle at the pixel coordinate returned by our SimpleGISconvert() function
- you'll notice that we must convert the population number to a string in order to use it in the turtle write() method. To do so, we use Python's built-in function, str()

In [9]:
for city in cities:
    pixel = convert(city[POINTS])
    t.up()
    t.goto(pixel)
    # Place a point for the city
    t.dot(10)
    # Label the city
    t.write(city[NAME] + ", Pop.: " + str(city[POP]), align="left")
    t.up()